In [4]:
import pandas as pd
import numpy as np
import os

DATA_DIR = "data"
l1_nrows = 1512
l2_nrows = 1248
n_horizons = 4
y_height = 3001

# Загружаем значения высот в узлах сетки для срезов L1 и L2

all_data_l1 = np.load(os.path.join(DATA_DIR, "all_data_L1.npy"))
all_data_l2 = np.load(os.path.join(DATA_DIR, "all_data_L2.npy"))

assert all_data_l1.shape == (l1_nrows, y_height), "Неправильный размер all_data_L1.npy"
assert all_data_l2.shape == (l2_nrows, y_height), "Неправильный размер all_data_L2.npy"

# Загружаем горизонты

l1_horizons_train = pd.read_csv(os.path.join(DATA_DIR, "L1_horizons_train.csv"))
l2_horizons_train = pd.read_csv(os.path.join(DATA_DIR, "L2_horizons_train.csv"))

assert l1_horizons_train.shape == (l1_nrows, n_horizons+1)
assert l2_horizons_train.shape == (l2_nrows, n_horizons+1)

# Нам необходимо предсказать значения второго горизонта среза L1 для всех x в промежутке 522, ..., 1451

# Посмотрим на пример посылки решения

sample_submission = pd.read_csv(os.path.join(DATA_DIR, "sample_submission.csv"))
my_submission = sample_submission.copy()

model = lambda x: 4
my_submission.y = my_submission.x.apply(model)

my_submission.to_csv("my_submission.csv", index=False)

In [5]:
import numpy as np

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense
from keras.utils import np_utils
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers.core import Dense
import tensorflow as tf


In [6]:
l2_horizons_train['hor_1'][57]
l1_horizons_train['hor_1'][59]
l1_horizons_train['hor_2'][52]



nan

In [7]:
all_data_l2[57][703]

3098290400.0

In [8]:
def is_none(num):
    if num is None:
        return 0.0
    else:
        return num

In [9]:
def add_output(output, hor_name):
    for i in range(59, 1225, 1):
        one_output = []
        one_output.append(i)
        one_output.append(is_none(l2_horizons_train[hor_name][i]))
        output.append(one_output)
    print(len(output))

In [10]:
def add_train(train, hor_name, start, stop):
    for i in range(start, stop, 1):
        one_train_set = []
        one_train_set.append(i)
        one_train_set.append(is_none(l2_horizons_train[hor_name][i]))
        one_train_set.append(is_none(all_data_l2[i+1][int(l2_horizons_train[hor_name][i])]))
        one_train_set.append(is_none(all_data_l2[i+1][int(l2_horizons_train[hor_name][i+1])]))
        one_train_set.append(is_none(all_data_l2[i+1][int(l2_horizons_train[hor_name][i-1])]))
        train.append(one_train_set)
        if i == 1:
            print(one_train_set)
    print(train[0])

In [11]:
train = []
add_train(train, 'hor_1', 58, 1224)
add_train(train, 'hor_3', 58, 1224)
add_train(train, 'hor_4', 58, 1224)
print(len(train))


[58, 703.0, 3604777500.0, 3604777500.0, 3604777500.0]
[58, 703.0, 3604777500.0, 3604777500.0, 3604777500.0]
[58, 703.0, 3604777500.0, 3604777500.0, 3604777500.0]
3498


In [12]:
all_data_l2[59][int(l2_horizons_train['hor_1'][70])]

3542290400.0

In [13]:
output = []
add_output(output, 'hor_1')
add_output(output, 'hor_3')
add_output(output, 'hor_4')

1166
2332
3498


In [14]:
test = []

for i in range(58, 1224, 1):
    one_train_set = []
    one_train_set.append(i)
    one_train_set.append(is_none(l2_horizons_train['hor_2'][i]))
    one_train_set.append(is_none(all_data_l2[i+1][int(l2_horizons_train['hor_2'][i])]))
    one_train_set.append(is_none(all_data_l2[i+1][int(l2_horizons_train['hor_2'][i+1])]))
    one_train_set.append(is_none(all_data_l2[i+1][int(l2_horizons_train['hor_2'][i-1])]))
    test.append(one_train_set)
print(test[0])

test_output = []
for i in range(59, 1225, 1):
    one_output = []
    one_output.append(i)
    one_output.append(is_none(l2_horizons_train['hor_2'][i]))
    test_output.append(one_output)
print(len(output))

[58, 814.59998, 2461823000.0, 2461823000.0, 2461823000.0]
3498


In [15]:
pred = []
add_train(pred, 'hor_2', 58, 521)

[58, 814.59998, 2461823000.0, 2461823000.0, 2461823000.0]


In [16]:
test_output = np.array(test_output)
test = np.array(test)
output = np.array(output)
train = np.array(train)

test_output /= 1000000000
test /= 1000000000
output /= 1000000000
train /= 1000000000

train

array([[5.80000000e-08, 7.03000000e-07, 3.60477747e+00, 3.60477747e+00,
        3.60477747e+00],
       [5.90000000e-08, 7.03200010e-07, 3.82076826e+00, 3.82076826e+00,
        3.82076826e+00],
       [6.00000000e-08, 7.03400020e-07, 4.04334003e+00, 4.04334003e+00,
        4.04334003e+00],
       ...,
       [1.22100000e-06, 9.68700010e-07, 3.57437133e+00, 3.57437133e+00,
        3.57437133e+00],
       [1.22200000e-06, 9.68799990e-07, 3.39701146e+00, 3.39701146e+00,
        3.39701146e+00],
       [1.22300000e-06, 9.68799990e-07, 3.30979226e+00, 3.30979226e+00,
        3.30979226e+00]])

In [17]:
train.shape

(3498, 5)

In [18]:
output = to_categorical(output)
test_output = to_categorical(test_output)

In [19]:
model = Sequential()
model.add(Dense(5, input_dim=5, activation='sigmoid'))
model.add(Dense(5, input_dim=5, activation='sigmoid'))
model.add(Dense(2))

In [21]:
optimizer = tf.keras.optimizers.RMSprop(0.0001)

model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 5)                 30        
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 12        
Total params: 72
Trainable params: 72
Non-trainable params: 0
_________________________________________________________________


In [23]:
history = model.fit(train, output, epochs=50, verbose=2, validation_data=(test, test_output))

Epoch 1/50
110/110 - 0s - loss: 2.2540 - mae: 1.4700 - mse: 2.2540 - val_loss: 2.0744 - val_mae: 1.4023 - val_mse: 2.0744
Epoch 2/50
110/110 - 0s - loss: 2.0816 - mae: 1.4124 - mse: 2.0816 - val_loss: 1.9112 - val_mae: 1.3452 - val_mse: 1.9112
Epoch 3/50
110/110 - 0s - loss: 1.9190 - mae: 1.3558 - mse: 1.9190 - val_loss: 1.7581 - val_mae: 1.2892 - val_mse: 1.7581
Epoch 4/50
110/110 - 0s - loss: 1.7650 - mae: 1.2998 - mse: 1.7650 - val_loss: 1.6145 - val_mae: 1.2344 - val_mse: 1.6145
Epoch 5/50
110/110 - 0s - loss: 1.6191 - mae: 1.2445 - mse: 1.6191 - val_loss: 1.4801 - val_mae: 1.1807 - val_mse: 1.4801
Epoch 6/50
110/110 - 0s - loss: 1.4813 - mae: 1.1898 - mse: 1.4813 - val_loss: 1.3540 - val_mae: 1.1279 - val_mse: 1.3540
Epoch 7/50
110/110 - 0s - loss: 1.3510 - mae: 1.1357 - mse: 1.3510 - val_loss: 1.2356 - val_mae: 1.0761 - val_mse: 1.2356
Epoch 8/50
110/110 - 0s - loss: 1.2281 - mae: 1.0821 - mse: 1.2281 - val_loss: 1.1246 - val_mae: 1.0251 - val_mse: 1.1246
Epoch 9/50
110/110 - 0s 

In [24]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [MPG]')
  plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
  plt.ylim([0,5])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$MPG^2$]')
  plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
  plt.ylim([0,20])
  plt.legend()
  plt.show()


plot_history(history)

NameError: name 'plt' is not defined

In [28]:
test_predictions = model.predict(pred)
test_predictions *= 1000

with open('data/sample_submission.csv', 'r') as old:
    new_sample = open('data/my_submission.csv', 'w')
    new_sample.write('x,y\n')
    old.readline()
    for i in test_predictions:
        x = old.readline().split(',')[0]
        new_sample.write('{},{:.2f}\n'.format(x, i[1]))
    for i in range(467):
        new_sample.write('{},{:.2f}\n'.format(984+i, 965.6))
# test_predictions
# print(pred[0])

In [1]:
1451 - 984

467